In [1]:
setwd("T:/PIgroup Nael NadifKasri/Sofia/2025/MEA_seq_patient/Analysis/RNA-seq/PRPS_integration_neurons")

In [ ]:
#one boxplot per cell line, compared to corresponding control levels, of the corresponding gene

In [2]:
library(tidyverse)
library(dplyr) 
library(ggplot2)
library(reshape2)
library(cowplot)  # for arranging plots and shared legend

library(tidyr)
library(gridExtra)


Warning message:
"package 'tidyverse' was built under R version 4.2.3"
Warning message:
"package 'tibble' was built under R version 4.2.3"
Warning message:
"package 'tidyr' was built under R version 4.2.3"
Warning message:
"package 'readr' was built under R version 4.2.3"
Warning message:
"package 'purrr' was built under R version 4.2.3"
Warning message:
"package 'dplyr' was built under R version 4.2.3"
Warning message:
"package 'stringr' was built under R version 4.2.3"
Warning message:
"package 'forcats' was built under R version 4.2.3"
Warning message:
"package 'lubridate' was built under R version 4.2.3"
── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ───────────────────────────────────────────────────────────────────

In [3]:
counts<-read.table("T:/PIgroup Nael NadifKasri/Sofia/2025/MEA_seq_patient/Analysis/RNA-seq/PRPS_integration_neurons/ruviii.2000.norm_voom.txt")
rownames(counts)<-counts$Gene
counts <- counts[, -ncol(counts)]
#replace "." to "-" for colnames
colnames_counts<-colnames(counts)
colnames_counts<-gsub("\\.", "-", colnames_counts)
colnames_counts<-str_replace_all(colnames_counts, "SMARCB1_KSS_CRISPR", "SMARCB1_KSS-CRISPR")
colnames(counts)<-colnames_counts

In [4]:
samples_info<-read.csv("T:/PIgroup Nael NadifKasri/Sofia/2025/MEA_seq_patient/Code/0.Data/RNAseq_data/samples_info.csv")
samples_info$Well_ID <- sub(".*_", "", samples_info$sample)
rownames(samples_info)<-samples_info$sample

In [5]:
color_vector_cell_line<- setNames(samples_info$color_2,samples_info$cell_line_v2) 
color_vector_cell_line <- color_vector_cell_line[!duplicated(names(color_vector_cell_line))]

In [8]:
# ----------------------------
# Prepare output
# ----------------------------
output_pdf <- "all_mutant_boxplots_gene_expression.pdf"
pdf(output_pdf, width = 8.27, height = 11.7)  # A4 size in inches

# CSV for statistics
stat_results <- data.frame(
  mutant = character(),
  gene = character(),
  control_mean = numeric(),
  mutant_mean = numeric(),
  t_statistic = numeric(),
  p_value = numeric(),
  stringsAsFactors = FALSE
)

# Store plots for arranging
plots_list <- list()

# Get mutant lines
mutant_lines <- samples_info %>% 
  filter(condition == "mutant") %>% 
  pull(group) %>% 
  unique()

for(mutant in mutant_lines) {
  
  # Get info for this mutant
  mut_info <- samples_info %>% filter(group == mutant)
  gene_of_interest <- unique(mut_info$gene)
  own_control <- unique(mut_info$own_control)
  
  # Collect mutant expression
  mut_samples <- mut_info$sample
  mut_expr <- counts[gene_of_interest, mut_samples, drop = FALSE] %>% 
    t() %>% as.data.frame()
  colnames(mut_expr) <- "expression"
  mut_expr$sample <- rownames(mut_expr)
  mut_expr$group <- mutant
  mut_expr$cell_line_v2 <- mut_info$cell_line_v2[match(mut_expr$sample, mut_info$sample)]
  
  # Collect control samples
  if(!is.na(own_control) && own_control != "All_controls") {
    ctrl_samples <- samples_info[samples_info$group == own_control,"sample"]
  } else {
    ctrl_samples <- samples_info[samples_info$condition == "control","sample"] 
  }
  
  ctrl_expr <- counts[gene_of_interest, ctrl_samples, drop = FALSE] %>% 
    t() %>% as.data.frame()
  colnames(ctrl_expr) <- "expression"
  ctrl_expr$sample <- rownames(ctrl_expr)
  ctrl_expr$group <- "control"
  ctrl_expr$cell_line_v2 <- samples_info$cell_line_v2[match(ctrl_expr$sample,samples_info$sample)]
  
  # Combine data
  plot_data <- bind_rows(mut_expr, ctrl_expr)
  plot_data$group <- ifelse(plot_data$group != "control", plot_data$cell_line_v2, "control")
  
  # ----------------------------
  # Compute statistics (t-test)
  # ----------------------------
  mutant_values <- plot_data$expression[plot_data$group != "control"]
  control_values <- plot_data$expression[plot_data$group == "control"]
  
  test <- t.test(mutant_values, control_values)
  
  stat_results <- rbind(stat_results, data.frame(
    mutant = unique(mut_expr$cell_line_v2),
    gene = gene_of_interest,
    control_mean = mean(control_values),
    mutant_mean = mean(mutant_values),
    t_statistic = test$statistic,
    p_value = test$p.value
  ))
  
  # ----------------------------
  # Generate plot with smaller text
  # ----------------------------
  p <- ggplot(plot_data, aes(x = group, y = expression)) +
    geom_boxplot(outlier.shape = NA) +
    geom_jitter(aes(fill = cell_line_v2), shape = 21, width = 0.2, size = 4, color = "black") +
    scale_fill_manual(values = color_vector_cell_line) +
    labs(title = paste0(gene_of_interest, " | Mutant: ", unique(mut_expr$cell_line_v2)),
         x = "", y = "log2(CPM)") +
    theme_minimal(base_size = 10) +  # smaller text
    theme(
      text = element_text(color = "black"),
       axis.text.x = element_text(color = "black", size = 8, angle = 45, hjust = 1),
    axis.text.y = element_text(color = "black", size = 8),
      axis.title = element_text(color = "black", size = 9),
      plot.title = element_text(color = "black", size = 10),
      legend.text = element_text(color = "black", size = 8),
      legend.title = element_text(color = "black", size = 9),
      legend.position = "none"
    )
  
  plots_list[[length(plots_list) + 1]] <- p
}

# ----------------------------
# Arrange all 9 plots in 3x3 grid on single page
# ----------------------------
grid.arrange(grobs = plots_list, ncol = 3, nrow = 3)

dev.off()

# ----------------------------
# Save statistics CSV
# ----------------------------
write.csv(stat_results, "mutant_vs_control_t_statistics.csv", row.names = FALSE)

pdf 
  2